# Hamza Tariq
# I210707
# Assignment 02

# Question # 01



In [76]:
import random
import time
import resource

def generate_population(pop_size):
    population = []
    for i in range(pop_size):
        mat = []
        for j in range(3):
            row = []
            for k in range(3):
                row.append(random.randint(1, 9))
            mat.append(row)
        population.append(mat)
    return population

def fitness_function(solution):
    target_sum = 15
    total_diff = 0

    # Calculate row sums
    for row in solution:
        total_diff += abs(sum(row) - target_sum)

    # Calculate column sums
    for i in range(3):
        col_sum = sum(solution[j][i] for j in range(3))
        total_diff += abs(col_sum - target_sum)

    # Calculate diagonal sums
    diag1_sum = sum(solution[i][i] for i in range(3))
    diag2_sum = sum(solution[i][2-i] for i in range(3))
    total_diff += abs(diag1_sum - target_sum) + abs(diag2_sum - target_sum)

    return total_diff

def selection(population, fitness_scores, num_parents):
    selected_parents = []
    for _ in range(num_parents):
        total_fitness = sum(fitness_scores)
        pick = random.uniform(0, total_fitness)
        current_fitness = 0
        for i in range(len(fitness_scores)):
            current_fitness += fitness_scores[i]
            if current_fitness > pick:
                selected_parents.append(population[i])
                break
    return selected_parents


num_generations = 2000

def crossover(parents):
    crossover_point = random.randint(1, len(parents[0]) - 1)
    child1 = []
    child2 = []
    for i in range(len(parents[0])):
        if i < crossover_point:
            child1.append(parents[0][i])
            child2.append(parents[1][i])
        else:
            child1.append(parents[1][i])
            child2.append(parents[0][i])
    return child1, child2

def mutate(state):
    # Introduce random mutation by swapping two numbers
    idx1 = random.randint(0, len(state) - 1)
    idx2 = random.randint(0, len(state) - 1)
    while idx2 == idx1:
        idx2 = random.randint(0, len(state) - 1)
    state[idx1], state[idx2] = state[idx2], state[idx1]
    return state

def print_matrix(matrix, n):
    for i in range(n):
        print(matrix[i])


def genetic_algorithm(population_size, num_generations, n):
    population = generate_population(population_size)
    for generation in range(num_generations):
        fitness_scores = [fitness_function(state) for state in population]
        if 0 in fitness_scores:
            idx = fitness_scores.index(0)
            print("Solution found ", ":", population[idx])
            print("Magic constant:", n * (n**2 + 1) // 2)
            print_matrix(population[idx], n)
            return population[idx]  # Solution found
        parents = selection(population, fitness_scores, 2)
        child1, child2 = crossover(parents)
        mutate(child1)
        mutate(child2)
        population.extend([child1, child2])
        population = sorted(population, key=lambda x: fitness_function(x))[:population_size]  # Keep top population_size states
    print("No solution found. ".format(num_generations))
    return None  # No solution found within num_generations

# Example usage
population_size = 100
n = 3  # Order of the magic square (3x3 in this case

# Time complexity
start_time = time.time()
solution = genetic_algorithm(population_size, num_generations, n)
end_time = time.time()
time_complexity = end_time - start_time

print("")
Sudoku_size_bytes = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / (1024 * 1024)

print("Memory usage of the Sudoku : {:.2f} MB".format(Sudoku_size_bytes))
print("Time Complexity:", time_complexity, "seconds")


Solution found  : [[5, 5, 5], [5, 5, 5], [5, 5, 5]]
Magic constant: 15
[5, 5, 5]
[5, 5, 5]
[5, 5, 5]

Memory usage of the Sudoku : 0.11 MB
Time Complexity: 0.5048136711120605 seconds


# Question # 02

In [77]:
import random
import heapq
import copy
import resource
import time

def solve_with_heuristics(bo):
    if not solve_ac3(bo):
        return False
    return solve_backtracking(bo)

def solve_backtracking(bo):
    find = find_empty(bo)
    if not find:
        return True
    else:
        row, col = find

    for i in range(1, 10):
        if valid(bo, i, (row, col)):
            bo[row][col] = i

            if solve_backtracking(bo):
                return True

            bo[row][col] = 0

    return False

def solve_ac3(bo):
    queue = []
    for i in range(9):
        for j in range(9):
            if bo[i][j] != 0:
                heapq.heappush(queue, (i, j))

    while queue:
        x, y = heapq.heappop(queue)
        if not revise(bo, x, y):
            continue
        if len(bo[x][y]) == 0:
            return False
        for i in range(9):
            if i != x and not (i, y) in queue:
                heapq.heappush(queue, (i, y))
        for j in range(9):
            if j != y and not (x, j) in queue:
                heapq.heappush(queue, (x, j))

    return True

def revise(bo, x, y):
    revised = False
    if isinstance(bo[x][y], list):
        for num in bo[x][y]:
            if not any(bo[x][j] == num and j != y for j in range(9)):
                bo[x][y].remove(num)
                revised = True
        if not bo[x][y]:
            return False
        for num in bo[x][y]:
            if not any(bo[i][y] == num and i != x for i in range(9)):
                bo[x][y].remove(num)
                revised = True
    return revised

def find_empty(bo):
    for i in range(len(bo)):
        for j in range(len(bo[0])):
            if bo[i][j] == 0:
                return (i, j)  # row, col
    return None

def valid(bo, num, pos):
    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False

    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y * 3, box_y * 3 + 3):
        for j in range(box_x * 3, box_x * 3 + 3):
            if bo[i][j] == num and (i, j) != pos:
                return False

    return True

def generate_sudoku():
    # Start with a completed Sudoku grid
    solved_board = [[0] * 9 for _ in range(9)]
    solve_with_heuristics(solved_board)  # Solve the board

    # Copy the solved board to work with
    puzzle = copy.deepcopy(solved_board)

    # Determine how many numbers to remove
    num_to_remove = random.randint(40, 55)  # Adjust the range as needed

    # Remove numbers to create the puzzle
    for _ in range(num_to_remove):
        row, col = random.randint(0, 8), random.randint(0, 8)
        while puzzle[row][col] == 0:
            row, col = random.randint(0, 8), random.randint(0, 8)
        puzzle[row][col] = 0

    return puzzle

def print_board(bo):
    for i in range(len(bo)):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - - ")

        for j in range(len(bo[0])):
            if j % 3 == 0 and j != 0:
                print(" | ", end="")

            if j == 8:
                print(bo[i][j])
            else:
                print(str(bo[i][j]) + " ", end="")

# Generate a random Sudoku puzzle
random_board = generate_sudoku()
print("Random Sudoku Puzzle:")
print_board(random_board)

# Solve the generated puzzle
start_time = time.time()
solve_with_heuristics(random_board)
end_time = time.time()

# Print the puzzle
print("\n-----------------------\n")
print("Solved Sudoku Puzzle:")
print_board(random_board)

# Measure space and time complexity of the solver.
Sudoku_size_bytes = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / (1024 * 1024)
time_complexity = end_time - start_time

print("\nMemory usage of the Sudoku : {:.2f} MB".format(Sudoku_size_bytes))
print("Time Complexity:", time_complexity, "seconds")


Random Sudoku Puzzle:
0 2 0  | 0 0 6  | 7 0 9
0 0 6  | 7 0 9  | 0 0 0
7 0 0  | 0 0 3  | 0 5 0
- - - - - - - - - - - - 
2 1 0  | 0 0 0  | 0 9 0
3 0 0  | 0 0 0  | 2 0 0
0 0 7  | 2 1 0  | 0 0 0
- - - - - - - - - - - - 
0 0 1  | 0 0 0  | 9 0 8
6 0 2  | 0 7 0  | 0 0 0
0 0 8  | 0 3 0  | 6 0 0

-----------------------

Solved Sudoku Puzzle:
1 2 3  | 4 5 6  | 7 8 9
4 5 6  | 7 8 9  | 1 2 3
7 8 9  | 1 2 3  | 4 5 6
- - - - - - - - - - - - 
2 1 4  | 3 6 5  | 8 9 7
3 6 5  | 8 9 7  | 2 1 4
8 9 7  | 2 1 4  | 3 6 5
- - - - - - - - - - - - 
5 3 1  | 6 4 2  | 9 7 8
6 4 2  | 9 7 8  | 5 3 1
9 7 8  | 5 3 1  | 6 4 2

Memory usage of the Sudoku : 0.11 MB
Time Complexity: 0.0016243457794189453 seconds
